### 2D Diffusion Equation :
$ -\frac{\partial}{\partial x}D(x,y)\frac{\partial}{\partial x}\phi(x,y) - -\frac{\partial}{\partial y}D(x,y\frac{\partial}{\partial y}\phi(x,y) + \Sigma_a(x,y)\phi(x,y) = S(x,y)$



### Descritized with FVM:
$\Sigma_{a,ij} = \phi_{i,j}(\Sigma_{a,i,j}V_{i,j} + \Sigma_{a,i+1,j}V_{i+1,j} + \Sigma_{a,i+1,j=1}V_{i+1,j=1} + \Sigma_{a,i,j+1}V_{i,j+1})$
$$ S_{ij} = S_{i,j}V_{i,j} + S_{i+1,j}V_{i+1,j} + S_{i+1,j+1}V_{i+1,j+1] + S_{i,j+1}V_{i,j+1} $$

In [4]:
import numpy as np

In [32]:
def it_gauss_sedidel(A, b, x, e):
    '''
    Ax = b with n iterations until error < e
    '''
    solutions = []
    L = np.tril(A)
    n = 1
    x = np.dot(np.linalg.inv(L), b - np.dot(A-L, x))
    err = 1
    solutions.append(list(x))    
    while err>e:
        x = np.dot(np.linalg.inv(L), b - np.dot(A-L, x))
        solutions.append(list(x))  
        err = np.linalg.norm(x-solutions[-2])/np.linalg.norm(x)
        n+=1
    return n, solutions[-1]
    


def DE_solver(x_pos, y_pos, D_mesh, abs_mesh, source_mesh, err_tol):
    """
    x_pos: position of material along x direction (n+1 by 1)
    y_pos: position of material along y direction (m+1 by 1)
    D_mesh: Diffusion constant distribution over meshes (m by n)
    abs_mesh: Absorption macroscopic cross section distribution over meshes (m by n)
    source_mesh: Fixed source distribution over meshes (m by n)
    """
    m = len(y_pos)-1
    n = len(x_pos)-1
    coeff_matrix = np.zeros(((m+1)*(n+1),(m+1)*(n+1)))
    solution = np.zeros(((m+1)*(n+1),1))
    
    #Absorption: Non corner or edge points    
    abs_list = solution.copy()
    for j in reversed(range(1,m-1)): #the position starts from bottom left corner
        for i in range(1,n-1):
            d1=np.abs(x_pos[j]-x_pos[i-1])
            e1=np.abs(y_pos[j]-y_pos[j+1])
            d2=np.abs(x_pos[i+1]-x_pos[i])
            e2=np.abs(y_pos[j-1]-y_pos[j])

            V1=0.25*d1*e1
            V2=0.25*d2*e1
            V3=0.25*d2*e2
            V4=0.25*d1*e2
            
            abs_value = abs_mesh[j,i-1]*V1+abs_mesh[j,i]*V2+abs_mesh[j-1,i]*V3+abs_mesh[j-1,i-1]*V4
            source_value = source_mesh[j,i-1]*V1+source_mesh[j,i]*V2+source_mesh[j-1,i]*V3+source_mesh[j-1,i-1]*V4
            abs_list[len(y_pos)*len(x_pos)-j*len(x_pos)+i] = abs_value
            solution[len(y_pos)*len(x_pos)-j*len(x_pos)+i]= source_value
            
    #Absorption: Reflecting Side Right Points
    for j in reversed(range(1,m-1)): 
        i=n
        d1=np.abs(x_pos[j]-x_pos[i-1])
        e1=np.abs(y_pos[j]-y_pos[j+1])
        e2=np.abs(y_pos[j-1]-y_pos[j])
        
        V1=0.25*d1*e1
        V4=0.25*d1*e2

        abs_value = abs_mesh[j,i-1]*V1+abs_mesh[j-1,i-1]*V4
        source_value = source_mesh[j,i-1]*V1+source_mesh[j-1,i-1]*V4
        abs_list[len(y_pos)*len(x_pos)-j*len(x_pos)+i] = abs_value
        solution[len(y_pos)*len(x_pos)-j*len(x_pos)+i]= source_value

    #Absorption: Reflecting Side top Points
    for i in range(1,n-1): 
        j=1
        d1=np.abs(x_pos[j]-x_pos[i-1])
        e1=np.abs(y_pos[j]-y_pos[j+1])
        d2=np.abs(x_pos[i+1]-x_pos[i])
        
        V1=0.25*d1*e1
        V2=0.25*d2*e1

        abs_value = abs_mesh[j,i-1]*V1+abs_mesh[j,i]*V2
        source_value = source_mesh[j,i-1]*V1+source_mesh[j,i]*V2
        abs_list[len(y_pos)*len(x_pos)-j*len(x_pos)+i] = abs_value
        solution[len(y_pos)*len(x_pos)-j*len(x_pos)+i]= source_value   
        
    #Absorption: Top Right
    i,j = n,1
    d1=np.abs(x_pos[j]-x_pos[i-1])
    e1=np.abs(y_pos[j]-y_pos[j+1])
    V1=0.25*d1*e1
    abs_value = abs_mesh[j,i-1]*V1
    source_value = source_mesh[j,i-1]*V1
    abs_list[len(y_pos)*len(x_pos)-j*len(x_pos)+i] = abs_value
    solution[len(y_pos)*len(x_pos)-j*len(x_pos)+i]= source_value
    
    Left=np.empty((n*(m+1),1))
    Right=np.empty((n*(m+1),1))
    Bottom=np.empty((m*(n+1),1))
    Top=np.empty((m*(n+1),1))
    Center=np.empty(((m+1)*(n+1),1))
    
    #Flux: Non corner or edge points
    for j in reversed(range(1,m-1)): #the position starts from bottom left corner
        for i in range(1,n-1):
            d1=np.abs(x_pos[j]-x_pos[i-1])
            e1=np.abs(y_pos[j]-y_pos[j+1])
            d2=np.abs(x_pos[i+1]-x_pos[i])
            e2=np.abs(y_pos[j-1]-y_pos[j])
            
            a_L=-(D_mesh[j,i-1]*e1+D_mesh[j-1,i-1]*e2)/(2*d1)
            a_R=-(D_mesh[j,i]*e1+D_mesh[j-1,i]*e2)/(2*d2)
            a_B=-(D_mesh[j,i-1]*d1+D_mesh[j,i]*d2)/(2*e1)
            a_T=-(D_mesh[j-1,i-1]*d1+D_mesh[j-1,i]*d2)/(2*e2)
            a_C=abs_list[len(y_pos)*len(x_pos)-j*len(x_pos)+i]-(a_L+a_R+a_B+a_T)
            
            Left[((m+1-j)*(n+1-1)+i-1)]=a_L
            Right[((m+1-j)*(n+1-1)+i)]=a_R
            Bottom[(m+1-j-1)*(n+1)+i]=a_B
            Top[(m+1-j-1)*(n+1)+i]=a_T
            Center[(m+1-j)*(n+1)+i]=a_C
    
    #Flux: Reflecting side top
    for j in reversed(range(1,m-1)): #the position starts from bottom left corner
        for i in range(1,n-1):
            d1=np.abs(x_pos[j]-x_pos[i-1])
            e1=np.abs(y_pos[j]-y_pos[j+1])
            #d2=bp.abs(x_pos[i+1]-x_pos[i])
            e2=np.abs(y_pos[j-1]-y_pos[j])
            
            a_L=-(D_mesh[j,i-1]*e1+D_mesh[j-1,i-1]*e2)/(2*d1)
            #a_R=-(D_mesh[j,i]*e1+D_mesh[j-1,i]*e2)/(2*d2)
            a_B=-(D_mesh[j,i-1]*d1+D_mesh[j,i]*d2)/(2*e1)
            a_T=-(D_mesh[j-1,i-1]*d1+D_mesh[j-1,i]*d2)/(2*e2)
            a_C=abs_list[len(y_pos)*len(x_pos)-j*len(x_pos)+i]-(a_L+a_R+a_B+a_T)
            
            Left[((m+1-j)*(n+1-1)+i-1)]=a_L
            #Right[((m+1-j)*(n+1-1)+i)]=a_R
            Bottom[(m+1-j-1)*(n+1)+i]=a_B
            Top[(m+1-j-1)*(n+1)+i]=a_T
            Center[(m+1-j)*(n+1)+i]=a_C    
    #Flux: Reflecting side right
    for j in reversed(range(1,m-1)): #the position starts from bottom left corner
        for i in range(1,n-1):
            d1=np.abs(x_pos[j]-x_pos[i-1])
            e1=np.abs(y_pos[j]-y_pos[j+1])
            d2=np.abs(x_pos[i+1]-x_pos[i])
            #e2=np.abs(y_pos[j-1]-y[j])
            
            a_L=-(D_mesh[j,i-1]*e1+D_mesh[j-1,i-1]*e2)/(2*d1)
            a_R=-(D_mesh[j,i]*e1+D_mesh[j-1,i]*e2)/(2*d2)
            a_B=-(D_mesh[j,i-1]*d1+D_mesh[j,i]*d2)/(2*e1)
            #a_T=-(D_mesh[j-1,i-1]*d1+D_mesh[j-1,i]*d2)/(2*e2)
            a_C=abs_list[len(y_pos)*len(x_pos)-j*len(x_pos)+i]-(a_L+a_R+a_B+a_T)
            
            Left[((m+1-j)*(n+1-1)+i-1)]=a_L
            Right[((m+1-j)*(n+1-1)+i)]=a_R
            Bottom[(m+1-j-1)*(n+1)+i]=a_B
            #Top[(m+1-j-1)*(n+1)+i]=a_T
            Center[(m+1-j)*(n+1)+i]=a_C   
    #Flux: Top right
    for j in reversed(range(1,m-1)): #the position starts from bottom left corner
        for i in range(1,n-1):
            d1=np.abs(x_pos[j]-x_pos[i-1])
            e1=np.abs(y_pos[j]-y_pos[j+1])
            #d2=bp.abs(x_pos[i+1]-x_pos[i])
            #e2=np.abs(y_pos[j-1]-y[j])
            
            a_L=-(D_mesh[j,i-1]*e1+D_mesh[j-1,i-1]*e2)/(2*d1)
            #a_R=-(D_mesh[j,i]*e1+D_mesh[j-1,i]*e2)/(2*d2)
            a_B=-(D_mesh[j,i-1]*d1+D_mesh[j,i]*d2)/(2*e1)
            #a_T=-(D_mesh[j-1,i-1]*d1+D_mesh[j-1,i]*d2)/(2*e2)
            a_C=abs_list[len(y_pos)*len(x_pos)-j*len(x_pos)+i]-(a_L+a_R+a_B+a_T)
            
            Left[((m+1-j)*(n+1-1)+i-1)]=a_L
            #Right[((m+1-j)*(n+1-1)+i)]=a_R
            Bottom[(m+1-j-1)*(n+1)+i]=a_B
            #Top[(m+1-j-1)*(n+1)+i]=a_T
            Center[(m+1-j)*(n+1)+i]=a_C

    for i in range((m+1)*(n+1)):
        coeff_matrix[i,i]=Center[i]
    
    for i in range(len(Top)):
        coeff_matrix[i,i+(n+1)*(m+1)-len(Top)]=Top[i]
        coeff_matrix[i+(n+1)*(m+1)-len(Bottom),i]=Bottom[i]

    A_Index=0
    V_Index=0
    for i in range((m+1)*(n+1)):
        skip = i%(n+1)
        if skip!=0: 
            coeff_matrix[A_Index,A_Index+1]=Right[V_Index]
            coeff_matrix[A_Index+1,A_Index]=Left[V_Index]
            A_Index+=1
            V_Index+=1
        else:
            A_Index+=1

    for i in range(len(solution)):
        if np.isnan(solution[i]):
            solution[i]=0
            coeff_matrix[i,:]= np.zeros((1,len(coeff_matrix[i,:])))
            coeff_matrix[i,i]=1

    guess = np.ones(((m+1)*(n+1),1))/np.linalg.norm(np.ones(((m+1)*(n+1),1)),2)
    flux_list, iterations = it_gauss_sedidel(coeff_matrix, solution, guess , err_tol)
    return flux_list
            

In [33]:
x_pos = np.array([0,4,8,12,16])
y_pos = np.array([0,4,8,12,16])
D_mesh = np.array([[1,0.25,1,0.25],
                  [0.25,1,0.25,1],
                  [1,0.25,1,0.25],
                  [0.25,1,0.25,1]])
abs_mesh = np.array([[0.5,2,0.5,0.2],
                  [0.2,0.5,0.2,0.5],
                  [0.5,0.2,0.5,0.2],
                  [0.2,0.5,0.2,0.5]])
source_mesh = np.array([[10,10,10,10],
                  [10,10,10,10],
                  [10,10,10,10],
                  [10,10,10,10]])
err_tol = 10**(-6)

In [34]:
DE_solver(x_pos, y_pos, D_mesh, abs_mesh, source_mesh, err_tol)

C:\Users\joann\Anaconda3\lib\site-packages\ipykernel\__main__.py:106: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\joann\Anaconda3\lib\site-packages\ipykernel\__main__.py:126: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\joann\Anaconda3\lib\site-packages\ipykernel\__main__.py:145: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\joann\Anaconda3\lib\site-packages\ipykernel\__main__.py:164: RuntimeWarning: divide by zero encountered in double_scalars


NameError: name 'A' is not defined